# imports

https://www.wrl.unsw.edu.au/news/coastsat-how-to-estimate-beach-slopes-using-satellite-imagery
https://www.wrl.unsw.edu.au/research/coastsat


In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
import time
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_earthcache_api, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects, SDS_earthcache_client

## setup timeline and AOL

In [2]:
#setup monthly pipelines
def fmt(x):
    if x < 10:
        return '0' + str(x)
    else:
        return str(x)
def monthly_pipes(startyear,endyear=2024,day=[0,0]):  
    starts = [] ; ends = []
    dates = range(startyear, endyear)  
    for y in dates:
        for m in range(1, 13):
            if day[0] == 0:
                starts.append(str(y) + '-' + fmt(m) + '-01')
            else:
                starts.append(str(y) + '-' + fmt(m) + '-' + fmt(day[0]))

            if day[1] == 0:
                if m == 2:
                    ends.append(str(y) + '-' + fmt(m) + '-28')
                elif m in [4, 6, 9, 11]:
                    ends.append(str(y) + '-' + fmt(m) + '-30')
                else:
                    ends.append(str(y) + '-' + fmt(m) + '-31')
            else:
                ends.append(str(y) + '-' + fmt(m) + '-' + fmt(day[1]))
    return starts, ends

In [3]:
# these coordinates are a rectangle around the island of Madhirivaadhoo
aoi = { "type": "Polygon", 
        "coordinates": [
      [
          [
            72.85318351534859,
            4.227805672204206,
            0
          ],
          [
            72.85339136125246,
            4.207331395520955,
            0
          ],
          [
            72.87337028624503,
            4.207517991124759,
            0
          ],
          [
            72.8731121458676,
            4.227805397834814,
            0
          ],
          [
            72.85318351534859,
            4.227805672204206,
            0
          ]
      ]
    ]
    }

# setup pipeline

In [4]:
key = input("please enter your API key for Earthcache:")

In [32]:
nb_client =SDS_earthcache_api.initialize_client(key,max_cost=100)
location_name = 'Ukulhas_'
resolution = 'medium'
interval = '5d'

d5431cc4-b43b-4f31-8db4-d1efe9e589c9


In [ ]:
nb_client.getOutputs()

'id': 'c31c1bea-a4be-11e7-8650-3ae5c7149ea7'

'name': 'All Optical Bands'

'description': 'All optical spectral bands in a product',

In [34]:
image_type = "a8fc3dde-a3e8-11e7-9793-ae4260ee3b4b" 

# test pipeline

In [35]:
# need to initialize the api first
#SDS_earthcache_api.initialize(key)
#create a set of dates so each pipeline is one month
starts,ends = monthly_pipes(2023)#,day=[2,0])
for i in range(0,1):#len(starts)):
    start_date = starts[i] 
    end_date = ends[i]
    pipeline_name = location_name + start_date + '_' + resolution
    print(pipeline_name + " ending: " + end_date)
    status, response = nb_client.calculatePrice(resolution, aoi, start_date, end_date,interval)
    if(status != 200):
        print(response['errors'])
    else:
        print(response['data']['archive'])
        print("balance: "+str(response['data']['prepurchase_balance']))
    #add 3 seconds of delay between each request
    time.sleep(3)

Ukulhas_2023-01-02_medium ending: 2023-01-31
{'interval_quantity': 6, 'price_km2': 2.5, 'minimum_area_km2': 0.1, 'interval_cost': 12.57, 'max_cost': 75.39}
balance: 3000.0


# run pipeline

In [36]:
for i in range(0,1):
    start_date = starts[i]
    end_date = ends[i]
    pipeline_name = location_name + start_date + '_' + resolution
    #SDS_earthcache_api.retrieve_images_earthcache(nb_client, pipeline_name, aoi, start_date, end_date, image_type,interval,resolution)
    time.sleep(3)

201 {'data': {'id': '2a37c5b7-f202-11ee-a186-09fa561e1e80', 'name': 'Ukulhas_2023-01-02_medium', 'start_date': '2023-01-02', 'end_date': '2023-01-31', 'aoi': {'type': 'Polygon', 'coordinates': [[[72.85318351534859, 4.227805672204206, 0], [72.85339136125246, 4.207331395520955, 0], [72.87337028624503, 4.207517991124759, 0], [72.8731121458676, 4.227805397834814, 0], [72.85318351534859, 4.227805672204206, 0]]]}, 'area_km2': 5.026146507863212, 'cloud_cover_percentage': 25, 'min_aoi_coverage_percentage': 75, 'interval': '5d', 'resolution_low': 30, 'resolution_high': 1.5, 'status': 'pending', 'tags': None, 'created_at': '2024-04-03T21:36:00.707559Z', 'updated_at': '2024-04-03T21:36:00.707559Z', 'max_cost': 100, 'result_delivery': {'max_latency': '0d', 'priorities': ['lowest_cost']}, 'output': {'id': 'a8fc3dde-a3e8-11e7-9793-ae4260ee3b4b', 'format': 'geotiff', 'mosaic': 'stitched'}}}
200 {'data': {'id': '2a37c5b7-f202-11ee-a186-09fa561e1e80', 'org_id': '001JR00000AfBY1YAN', 'name': 'Ukulhas_20

# check results and download

In [37]:
# after about 20 minutes, make sure the status is ready
for i in range(0,1):#len(starts)):
    start_date = starts[i]
    end_date = ends[i]
    pipeline_name = location_name + start_date + '_' + resolution
    status,res = SDS_earthcache_api.checkStatus(nb_client, pipeline_name)
    print(status,res)

200 {'data': [{'id': '45bba2a6-0e36-47d8-b053-c5683e5ac7c6', 'created_at': '2024-04-03T21:36:18.024367+0000', 'updated_at': '2024-04-03T21:36:42.421768+0000', 'pipeline_id': '2a37c5b7-f202-11ee-a186-09fa561e1e80', 'output_id': 'a8fc3dde-a3e8-11e7-9793-ae4260ee3b4b', 'status': 'processing', 'message': 'Retrieval - starting', 'interval': {'start_date': '2023-01-02', 'end_date': '2023-01-06'}, 'total_interval_cost': None, 'overall_metadata': {'scene_height': None, 'scene_width': None, 'filled_area_km2': None, 'filled_area_percentage_of_aoi': None, 'cloud_cover_percentage': None, 'cloud_cover_percentage_of_aoi': None, 'visible_area_km2': None, 'visible_area_percentage': None, 'visible_area_percentage_of_aoi': None}, 'results': [], 'alternate_search_results': []}, {'id': '2d98f451-c059-45a7-9d7f-f8afdf753eed', 'created_at': '2024-04-03T21:36:17.818866+0000', 'updated_at': '2024-04-03T21:37:06.444470+0000', 'pipeline_id': '2a37c5b7-f202-11ee-a186-09fa561e1e80', 'output_id': 'a8fc3dde-a3e8-11

In [ ]:
# once the status is good, run this cell to download the images
images = []
for i in range(0,1):#len(starts)):
    start_date = starts[i]
    pipeline_name = location_name + start_date + '_' + resolution
    status,res = SDS_earthcache_api.checkStatus(nb_client, pipeline_name)
    if(status == 200):
        images.append(SDS_earthcache_api.download_images(nb_client, pipeline_name))

In [43]:
images[0][0][0]

'data/Ukulhas_2023-01-02_medium\\45bba2a6-0e36-47d8-b053-c5683e5ac7c6\\20230103_043520/SKYWATCH_PLSD_PS_20230103T0435_TC_Tile_0_0_ec9f.tif'

# visualize sample

In [44]:
# this function visualizes the first image
SDS_earthcache_api.view_first_image(images[0])

AttributeError: 'NoneType' object has no attribute 'ReadAsArray'